# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [4]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,Hotel_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Luanda,AO,78.80,Clouds,-8.8368,13.2343,Hotel Forum
1,2,Dwarka,IN,77.23,Clear,22.2394,68.9678,The Dwarika Hotel
2,5,Matale,LK,72.55,Clouds,7.4721,80.6234,The Grand Mountain Hotel
3,13,Maxixe,MZ,75.02,Clouds,-23.8597,35.3472,Hotel Herry
4,16,Rikitea,PF,73.17,Clear,-23.1203,-134.9692,Chez Bianca & Benoit


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [6]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", frame_width=700, frame_height=500,
    size = "Max Temp", scale = 0.5, color = 'City',
    hover_cols = ["City", "Hotel Name", "Country", "Current Description"]
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [18]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"]=="Acapulco"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Acapulco"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Puerto Escondido"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Tres Picos"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Jalpan"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [19]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,Hotel_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
72,278,Acapulco,MX,80.42,Clear,16.8634,-99.8901,Hotel del Valle
173,682,Puerto Escondido,MX,79.63,Clouds,15.8500,-97.0667,Hotel Barlovento
80,306,Tres Picos,MX,81.88,Clouds,15.8667,-93.5333,Hotel Tres Picos
119,452,Jalpan,MX,75.25,Clouds,21.2333,-99.4833,Posada Karina
72,278,Acapulco,MX,80.42,Clear,16.8634,-99.8901,Hotel del Valle


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [20]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()

# Display sample data
waypoints_df

,Lat,Lng
72,16.8634,-99.8901
173,15.8500,-97.0667
80,15.8667,-93.5333
119,21.2333,-99.4833
72,16.8634,-99.8901


## Use GeoViews to create map that shows the four cities in the itinerary

In [23]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="EsriNatGeo", frame_width=700, frame_height=500,
    size = "Max Temp", scale = 2, color = 'City',
    hover_cols = ["City", "Hotel Name", "Country", "Current Description"])

In [24]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [25]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [26]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'c2616e101df44975be5abcd65bcdad7a',
 'waypoints': '16.8634,-99.8901|15.85,-97.0667|15.8667,-93.5333|21.2333,-99.4833|16.8634,-99.8901'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [28]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-99.8901, 16.8634], 'original_index': 0},
     {'location': [-97.0667, 15.85], 'original_index': 1},
     {'location': [-93.5333, 15.8667], 'original_index': 2},
     {'location': [-99.4833, 21.2333], 'original_index': 3},
     {'location': [-99.8901, 16.8634], 'original_index': 4}],
    'units': 'metric',
    'distance': 2873353,
    'distance_units': 'meters',
    'time': 118420.119,
    'legs': [{'distance': 399337,
      'time': 17997.738,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 30,
        'time': 3.676,
        'instruction': {'text': 'Drive south on Calle Vicente Guerrero.'}},
       {'from_index': 1,
        'to_index': 3,
        'distance': 44,
        'time': 11.016,
        'instruction': {'text': 'Turn left onto Avenida Sonora.'}},
       {'from_index': 3,
        'to_index': 5,
        'distance': 208,
        'time': 27.305,
        'i

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [37]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates'][0]
legs

[[-99.890005, 16.863433],
 [-99.889908, 16.863173],
 [-99.889821, 16.863208],
 [-99.889529, 16.863326],
 [-99.888865, 16.861719],
 [-99.888795, 16.861596],
 [-99.886922, 16.862524],
 [-99.886252, 16.862814],
 [-99.885929, 16.862932],
 [-99.885241, 16.863157],
 [-99.884813, 16.863321],
 [-99.884299, 16.863477],
 [-99.883771, 16.863641],
 [-99.882228, 16.864081],
 [-99.878291, 16.865203],
 [-99.878176, 16.865237],
 [-99.877718, 16.865377],
 [-99.874925, 16.866225],
 [-99.87454, 16.866349],
 [-99.87436, 16.866424],
 [-99.874119, 16.866558],
 [-99.873876, 16.86671],
 [-99.873623, 16.866917],
 [-99.873231, 16.86724],
 [-99.872935, 16.867548],
 [-99.872765, 16.867761],
 [-99.87265, 16.867969],
 [-99.872544, 16.868213],
 [-99.872501, 16.868415],
 [-99.872461, 16.868711],
 [-99.872373, 16.870575],
 [-99.872359, 16.870741],
 [-99.87227, 16.870843],
 [-99.872043, 16.871056],
 [-99.871888, 16.871172],
 [-99.871716, 16.871264],
 [-99.871537, 16.871337],
 [-99.871359, 16.871384],
 [-99.871166, 16.8

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [45]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for i in range(len(legs)):
    longitude.append(legs[i][0])
    latitude.append(legs[i][1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [49]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,-99.890005,16.863433
1,-99.889908,16.863173
2,-99.889821,16.863208
3,-99.889529,16.863326
4,-99.888865,16.861719


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [55]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df)

In [59]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map, route_path

(:Overlay
    .Tiles.I  :Tiles   [x,y]
    .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description),
 :Path   [longitude,latitude])